# Machine Learning Best Practices Assignment

In [1]:
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

### Import the [Pima Indians Diabetes data set](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/pima_indians_diabetes.csv).

In [2]:
data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/pima_indians_diabetes.csv')

In [3]:
data.head()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Split the data into training and test sets, with the target variable being the outcome column.

In [4]:
y = data.outcome
X = data.drop(columns='outcome')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### Train a Random Forest Classifier on the data without doing any transformations and print a classification report.

This will provide us with a basis for comparison.

In [5]:
model = RandomForestClassifier(random_state=1)

In [6]:
rf = model.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86       123
           1       0.78      0.65      0.71        69

    accuracy                           0.81       192
   macro avg       0.80      0.77      0.78       192
weighted avg       0.80      0.81      0.80       192



### Reduce the data down to 3 dimensions using PCA. Then do the train-test split, fit the model, and print a classification report.

Compare these results to the previous ones.

In [7]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.25, random_state=1)
model = RandomForestClassifier(random_state=1)
rf = model.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       123
           1       0.75      0.61      0.67        69

    accuracy                           0.79       192
   macro avg       0.78      0.75      0.76       192
weighted avg       0.78      0.79      0.78       192



### Fit the model and print a classification report again, but this time, perform the train-test split before you transform the data using PCA.

Compare these results to the previous ones.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


model = RandomForestClassifier(random_state=1)
rf = model.fit(X_train_pca, y_train)

y_test_pred = rf.predict(X_test_pca)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       123
           1       0.74      0.58      0.65        69

    accuracy                           0.78       192
   macro avg       0.77      0.73      0.74       192
weighted avg       0.77      0.78      0.77       192



### Using the Random Forest Classifier, perform 10-fold cross validation on the training set and print the mean cross validation score.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model = RandomForestClassifier(random_state=1)
rf = model.fit(X_train_pca, y_train)

scores = cross_val_score(rf, X_train_pca, y_train, cv=10, n_jobs=-1)
print('Mean score: %.3f' %(scores.mean()))

Mean score: 0.700


### Create a pipeline with a PCA step and a Random Forest Classifier step. Perform the train-test split again, fit the pipeline, and then generate a classification report.

Compare these results to the previous ones.

In [12]:
pipe = Pipeline([
             ('pca', PCA()),
             ('rf', RandomForestClassifier())    
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86       123
           1       0.78      0.65      0.71        69

    accuracy                           0.81       192
   macro avg       0.80      0.77      0.78       192
weighted avg       0.80      0.81      0.80       192



### Using the pipeline you built, perform 10-fold cross validation on the training set and print the mean cross validation score.

How does this score compare to the previous one?

In [13]:
scores = cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=-1)
print('Mean score: %.3f' %(scores.mean()))

Mean score: 0.738


### Use GridSearchCV to find the optimal set of parameters from the ones below.

- PCA Number of Components: 2, 3, 4, 5, 6, 7, 8
- Random Forest Number of Estimators: 10, 20, 50, 100, 200

In [21]:
pipe = Pipeline([
             ('pca', PCA()),
             ('rf', RandomForestClassifier())    
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

parameters = {
    'pca__n_components': [2, 3, 4, 5, 6, 7, 8],
    'rf__n_estimators': [10, 20, 50, 100, 200],
    }

search = GridSearchCV(pipe, parameters, cv=10)
search.fit(X_train, y_train)

print("Best parameter CV score: %0.3f" % search.best_score_)
print("Best parameters:", search.best_params_)

Best parameter CV score: 0.753
Best parameters: {'pca__n_components': 6, 'rf__n_estimators': 100}


### Using the best estimator pipeline from above, fit the pipeline to the training set and generate a classification report showing the results.

Compare these results to the previous ones.

In [22]:
pipeline = search.best_estimator_

pipeline.fit(X_train, y_train)
y_test_preds = pipeline.predict(X_test)

print(classification_report(y_test, y_test_preds))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       123
           1       0.69      0.61      0.65        69

    accuracy                           0.76       192
   macro avg       0.74      0.73      0.73       192
weighted avg       0.76      0.76      0.76       192



### Fit the best estimator pipeline to the entire data set and save your model to disk using pickle.

In [23]:
import pickle

pipeline.fit(X, y)

with open('model.pkl', 'wb') as f:
  pickle.dump(pipeline, f)

### Load the model you saved to disk, create a copy of the features in the data, and generate a set of predictions for those features. 

In [28]:
data_copy = X.copy()

with open('model.pkl', 'rb') as f:
  loaded_pipe = pickle.load(f)

preds = loaded_pipe.predict(data_copy)